In [1]:
import lsdtopytools as lsd, glob
import os

/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/geoconvtools.py:79: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython = False)
/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/numba_tools.py:73: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython=False,

In [2]:
# Every time you change something within the code cell, run it once more to update the variable.

BOUNDS = [39.7299, -77.5886, 36.9909, -81.6319] # In decimal coordinate form.  Format: [N, E, S, W].  Select the smallest area possible in your analyses for maximum resoultion!  

API_KEY = '9086887f175d33aa72eda767f5b1e9cd' # Get your API key from opentopography.org.  Students get higher resolution data!

DEM_DATASET = 'USGS30m' # Options are USGS30m, USGS10m, & USGS1m.  Be warned!  As resolution goes up, available download area goes down.

PROJ_TITLE = 'wv_tot' # Create a succinct name with no spaces or leading digits to represent your project file for future exports.

PATH = os.getcwd()

In [3]:
names = glob.glob(f'{PATH}/{PROJ_TITLE}/reproj/*.tiff')
names = [path.split(sep='/')[-1] for path in names]
names

['Carter_Run-Rappahannock_River_reproj.tiff',
 'Naked_Creek-South_Fork_Shenandoah_River_reproj.tiff',
 'Hawksbill_Creek-South_Fork_Shenandoah_River_reproj.tiff',
 'Poni_River_reproj.tiff',
 'Mine_Run-Rapidan_River_reproj.tiff',
 'Brush_Creek_reproj.tiff',
 'Evitts_Creek_reproj.tiff',
 'Savage_River_reproj.tiff',
 'Piney_Creek_reproj.tiff',
 'Georges_Creek_reproj.tiff',
 'Thumb_Run-Rappahannock_River_reproj.tiff',
 'Trading_Run-North_Branch_Potomac_River_reproj.tiff',
 'Wills_Creek_reproj.tiff',
 'New_Creek-North_Branch_Potomac_River_reproj.tiff',
 'Sand_Fork_reproj.tiff',
 'Patterson_Creek_reproj.tiff',
 'Little_Walker_Creek-Walker_Creek_reproj.tiff',
 'Dunkard_Creek_reproj.tiff',
 'Stony_River-North_Branch_Potomac_River_reproj.tiff',
 'Kimberling_Creek-Walker_Creek_reproj.tiff',
 'Leading_Creek_reproj.tiff',
 'Right_Fork_Spring_Creek-Spring_Creek_reproj.tiff',
 'Right_Fork_Steer_Creek-Steer_Creek_reproj.tiff',
 'West_Fork_Little_Kanawha_River_reproj.tiff',
 'Upper_Little_Kanawha_River

In [ ]:
def make_ksn_df(dem_name, path=f'{PATH}/{PROJ_TITLE}/reproj/'):
    name = dem_name[:-5]
    mydem = lsd.LSDDEM(path, dem_name)
    print('Raster Loaded')
    mydem.PreProcessing()
    print('PreProcessed')
    mydem.CommonFlowRoutines()
    print('Flow routines got')
    mydem.ExtractRiverNetwork()
    print('River Network Extracted')
    mydem.DefineCatchment(method='main_basin')
    print('Catchment defined')
    mydem.GenerateChi()
    print('Chi generated')
    mydem.ksn_MuddEtAl2014(target_nodes=70, n_iterations=60, skip=1, minimum_segment_length=10, sigma=2,  nthreads = 1, reload_if_same = False)
    ksn_df = mydem.df_ksn
    mydem.knickpoint_extraction(lambda_TVD = "auto", combining_window = 30, window_stepped = 80, n_std_dev = 7)
    knickpoint_df = mydem.df_knickpoint
    knickpoint_df.to_csv(f'{PATH}/{PROJ_TITLE}/kp_csvs/{name}_knickpoints.csv')
    ksn_df.to_csv(f'{PATH}/{PROJ_TITLE}/ksn_csvs/{name}_ksn.csv')

In [7]:
os.mkdir(f'{PATH}/{PROJ_TITLE}/ksn_csvs')
os.mkdir(f'{PATH}/{PROJ_TITLE}/kp_csvs')

FileExistsError: [Errno 17] File exists: '/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/wv_tot/ksn_csvs'

In [ ]:
for name in names:
    make_ksn_df(name)